In [1]:
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
import torch
import languagemodel
import embedding
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL = languagemodel.BERT(DEVICE, 'bert-base-cased', 32)

Language model 'bert-base-cased'initialized (batchsize = 32) on cpu.


In [2]:
def get_pmi(sentences, n_sentences='all', verbose=False):
    '''get estimates get scores for n (default all) sentences'''

    savez_dict = dict()

    if n_sentences == 'all':
        n_sentences = len(sentences)

    for i, sentence in enumerate(tqdm(sentences[:n_sentences], leave=False)):
        # get a pmi matrix and a pseudo-logprob for the sentence
        pmi_matrix, pseudo_loglik = MODEL.ptb_tokenlist_to_pmi_matrix(
            sentence, add_special_tokens=True, verbose=verbose,
            pad_left=None, pad_right=None)
        if verbose:
            tqdm.write(f"pmi for sentence {i}\n{sentence}")
            tqdm.write(f"pseudo_loglik: {pseudo_loglik}")
            tqdm.write(f"{pmi_matrix}")

        savez_dict[str(i)] = pmi_matrix
    return savez_dict

In [3]:
sentences=[
    "Each of the four earthly elements has its natural place .",
    "By soul he simply meant the form of a living being .",
    "Increasingly , women are contributing more to household income than men .",
    "Multiplying all observations by a constant does not alter significance ."
]
get_pmi(sentences)

  3%|▎         | 3/102 [00:13<07:05,  4.30s/it]

KeyboardInterrupt: 